# FFM

Eileen Zhang 2020/8/21

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
from tensorflow.keras.layers import Layer,Dense,Embedding

In [3]:
class FieldAwareFactorizationMachine(Layer):
    import tensorflow as tf
    """
    A keras implementation of Field-aware Factorization Machine.

    Reference:
        Y Juan, et al. Field-aware Factorization Machines for CTR Prediction, 2015.
    """
    
    def __init__(self, field_dim_groups, embed_dim):
        """
        :param x: field_dim_groups features group(the max_len for features)eg: [[10,20,40],[20,30,10],[5,6]]]``
                  num_field_groups : [0,3,6]``
        
        """        
        super().__init__()
        self.field_dim_groups = field_dim_groups
        self.features_len = np.sum([len(x) for x in field_dim_groups])
        self.num_field_groups = np.cumsum([0] + [len(x) for x in field_dim_groups])[:-1]
        #for irregular list [[10,20,40],[20,30,10],[5,6]]]
        self.maxlen = np.sum(np.sum(field_dim_groups))
        self.embeddings = Embedding(self.maxlen, embed_dim)
        self.offsets = tf.constant(np.expand_dims(np.array((0, *np.cumsum([x for y in field_dim_groups for x in y])[:-1]), dtype=np.float32),0))

    def build(self, input_shape):    
        super().build(input_shape)
        
    def call(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + self.offsets
        x = self.embeddings(x)
        ix = []
        for i in range(len(self.field_dim_groups) - 1):           
            for j in range(self.num_field_groups[i], self.num_field_groups[i + 1]):
                ix += [x[:, j, : ] * x[:, k, :] for k in range(self.num_field_groups[i + 1], self.features_len)]
        ix = tf.stack(ix)
        ix = tf.transpose(ix, perm=[1, 0, 2])
        return tf.concat([x, ix],1)

In [4]:
test = tf.constant([[0.,1.,2.,3.,4.],[10.,20.,30.,40.,50.]])

In [5]:
class FieldAwareFactorizationMachineModel(keras.models.Model):
    import tensorflow as tf
    """
    A keras implementation of Field-aware Factorization Machine.

    Reference:
        Y Juan, et al. Field-aware Factorization Machines for CTR Prediction, 2015.
    """

    def __init__(self, field_dim_groups, embed_dim):
        super().__init__()
        self.embed_dim = embed_dim
        self.ffm = FieldAwareFactorizationMachine(field_dim_groups, embed_dim)
        self.dense = Dense(1)
        self.fc = Dense(1)
        
    def call(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = self.ffm(x)
        x = tf.squeeze(self.dense(x),-1)
        x = self.fc(x)
        return x

In [6]:
model = FieldAwareFactorizationMachineModel([[10,20,40],[50,51]],3)

In [7]:
test

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[ 0.,  1.,  2.,  3.,  4.],
       [10., 20., 30., 40., 50.]], dtype=float32)>

In [8]:
model(test)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.02888147],
       [-0.04758702]], dtype=float32)>

In [9]:
model.summary()

Model: "field_aware_factorization_machine_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
field_aware_factorization_ma multiple                  513       
_________________________________________________________________
dense (Dense)                multiple                  4         
_________________________________________________________________
dense_1 (Dense)              multiple                  12        
Total params: 529
Trainable params: 529
Non-trainable params: 0
_________________________________________________________________
